<font color=red>
Spark_Version:2.0.0<br/>
Python_Version:Python 3.5.2 | Anaconda4.1.1(64-bit)<br/>
Jupyter_Version:4.2.1</br>
System:Ubuntu 16.04 LTS(64-bit)
</font>

In [1]:
import platform
print("Spark_Version:",sc.version)
print("Python_Version:",platform.python_version())
print("System:",platform.system())

Spark_Version: 2.0.0
Python_Version: 3.5.2
System: Linux


## Creating DataFrames from RDDs
The following code creates an RDD from a list of colors followed by a collection of tuples containing the color name and its length. It creates a DataFrame using the `toDF` method to convert the RDD into a DataFrame. The `toDF` method takes a list of column labels as an optional argument:

In [2]:
#Create a list of colours
colors = ['white', 'green', 'yellow', 'red', 'brown', 'pink']
#Distribute a local collection to form an RDD
#Apply map function on that RDD to get another RDD containing colour,length tuples
color_df = sc.parallelize(colors).map(lambda x: (x,len(x))).toDF(['color', 'length'])

In [3]:
color_df

DataFrame[color: string, length: bigint]

In [4]:
color_df.dtypes

[('color', 'string'), ('length', 'bigint')]

In [5]:
color_df.show()

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
| brown|     5|
|  pink|     4|
+------+------+



## Creating DataFrames from JSON
JavaScript Object Notation, or JSON, is a language-independent, self-describing, lightweight data-exchange format. JSON has become a popular data exchange format and has become ubiquitous. In addition to JavaScript and RESTful interfaces, databases such as MySQL have accepted JSON as a data type and MongoDB stores all data as JSON documents in binary form. Conversion of data to and from JSON is essential for any modern data analysis workflow. The Spark DataFrame API lets developers convert JSON objects into DataFrames and vice versa. Let's have a close look at the following examples for a better understanding: